(sec:T2:ejemplo_convolucion)=
### Convolución continua

Un ejemplo de convolución para el tema 2

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath(".."))

from utils.plot_helpers import style_math_axes

In [4]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Label, Span, LabelSet
from bokeh.io import output_notebook

# Importamos tus helpers
from utils.plot_helpers import style_math_axes, add_math_ticks

output_notebook()

# ==========================================
# 1. DEFINICIÓN DE SEÑALES
# ==========================================
tau_min, tau_max = -3.0, 6.0
num_points = 1000
tau_vals = np.linspace(tau_min, tau_max, num_points)
dtau = tau_vals[1] - tau_vals[0]

def signal_x(t): return np.where((t >= 0) & (t <= 2), 1.0, 0.0) 
def signal_h(t): return np.where((t >= 0) & (t <= 1), 1.0, 0.0) 

x_data = signal_x(tau_vals)

# Convolución analítica
y_full = np.convolve(x_data, signal_h(tau_vals), mode='full') * dtau
t_y_start = 2 * tau_min
t_y_vals = np.linspace(t_y_start, t_y_start + (len(y_full)-1)*dtau, len(y_full))
mask_y = (t_y_vals >= tau_min) & (t_y_vals <= tau_max)
y_display_t = t_y_vals[mask_y]
y_display_val = y_full[mask_y]

t_init = -1.0

# ==========================================
# 2. FUENTES DE DATOS
# ==========================================

source_signals = ColumnDataSource(data=dict(
    tau=tau_vals, x=x_data, h_shifted=np.zeros_like(tau_vals)
))

# >>> NUEVO: Ticks dinámicos en el eje <<<
# Usamos 'y=0' para que estén en el eje. 
# Quitamos los $$ y usamos texto normal que estilizaremos como itálico.
source_moving_ticks = ColumnDataSource(data=dict(
    x=[t_init, t_init - 1.0], 
    y=[0, 0], 
    text=["t", "t-1"] 
))

source_prod = ColumnDataSource(data=dict(tau=tau_vals, prod=np.zeros_like(tau_vals)))
source_res = ColumnDataSource(data=dict(t=y_display_t, y=y_display_val))
source_dot = ColumnDataSource(data=dict(t=[t_init], y=[0]))

# ==========================================
# 3. GRÁFICOS
# ==========================================
plot_width = 600
plot_height = 240
RED_COLOR = "#d62728"

# --- GRÁFICA 1 ---
p1 = figure(width=plot_width, height=plot_height, title="1. Flip & Slide: x(τ) y h(t - τ)")
style_math_axes(p1, x_range=(tau_min, tau_max), y_range=(-0.5, 1.6), xlabel=r"$$\tau$$", ylabel="Amplitud")

# Ticks Estáticos (Negros)
add_math_ticks(p1, xticks=[0, 1, 2, 3], xtick_labels=["0", "1", "2", "3"])

# >>> TICKS DINÁMICOS (Rojos) <<<
# 1. La rayita del tick (Segmento vertical pequeño cruzando el eje)
p1.segment(x0='x', y0=-0.1, x1='x', y1=0.1, source=source_moving_ticks, color=RED_COLOR, line_width=2)

# 2. La etiqueta del tick (t y t-1)
# y_offset=-20 las baja para alinearlas con los números negros
labels_dyn = LabelSet(x='x', y=0, text='text', source=source_moving_ticks, 
                      text_align='center', text_baseline='middle',
                      y_offset=-20, text_color=RED_COLOR, 
                      text_font_style='italic', text_font_size="11pt")
p1.add_layout(labels_dyn)

# Dibujo de señales
p1.line('tau', 'x', source=source_signals, color="#1f77b4", line_width=2, legend_label="x(τ)")
p1.varea(x='tau', y1='x', y2=0, source=source_signals, fill_color="#1f77b4", fill_alpha=0.1)
p1.line('tau', 'h_shifted', source=source_signals, color=RED_COLOR, line_width=2, line_dash="dashed", legend_label="h(t - τ)")

p1.legend.location = "top_right"
p1.legend.background_fill_alpha = 0


# --- GRÁFICA 2 ---
p2 = figure(width=plot_width, height=plot_height, title="2. Producto: x(τ) · h(t - τ)")
style_math_axes(p2, x_range=(tau_min, tau_max), y_range=(-0.5, 1.5), xlabel=r"$$\tau$$", ylabel="Producto")
add_math_ticks(p2, xticks=[0, 1, 2, 3], xtick_labels=["0", "1", "2", "3"])

p2.varea(x='tau', y1='prod', y2=0, source=source_prod, fill_color="purple", fill_alpha=0.4)
p2.line('tau', 'prod', source=source_prod, color="purple", line_width=1.5)


# --- GRÁFICA 3 ---
p3 = figure(width=plot_width, height=plot_height, title="3. Integral Resultante: y(t)")
style_math_axes(p3, x_range=(tau_min, tau_max), y_range=(-0.5, 2.5), xlabel="t", ylabel="y(t)")
add_math_ticks(p3, xticks=[0, 1, 2, 3], xtick_labels=["0", "1", "2", "3"])

p3.line('t', 'y', source=source_res, color="#2ca02c", line_width=2.5, alpha=0.6)
p3.circle('t', 'y', source=source_dot, color="#2ca02c", size=10, line_color="black")
span_t = Span(location=t_init, dimension='height', line_color='black', line_dash='dotted', line_width=1)
p3.add_layout(span_t)


# ==========================================
# 4. INTERACTIVIDAD
# ==========================================
slider_t = Slider(start=-2, end=5, value=t_init, step=0.05, title=r"Tiempo t")

callback = CustomJS(
    args=dict(source_sig=source_signals, 
              source_prod=source_prod, 
              source_dot=source_dot,
              source_res=source_res,
              source_ticks=source_moving_ticks, # <--- Source de ticks rojos
              span_t=span_t,
              slider=slider_t),
    code="""
    const t_curr = slider.value;
    
    const tau = source_sig.data['tau'];
    const x = source_sig.data['x'];
    const h = source_sig.data['h_shifted'];
    const prod = source_prod.data['prod'];
    
    const h_start = t_curr - 1.0; 
    const h_end = t_curr;
    
    // 1. Actualizar señales
    for (let i = 0; i < tau.length; i++) {
        let val_tau = tau[i];
        if (val_tau >= h_start && val_tau <= h_end) {
            h[i] = 1.0;
        } else {
            h[i] = 0.0;
        }
        prod[i] = x[i] * h[i];
    }
    source_sig.change.emit();
    source_prod.change.emit();
    
    // 2. Actualizar Posición Ticks (t y t-1)
    source_ticks.data['x'][0] = t_curr;       // t
    source_ticks.data['x'][1] = t_curr - 1.0; // t-1
    source_ticks.change.emit();

    // 3. Actualizar Resultado
    const res_t = source_res.data['t'];
    const res_y = source_res.data['y'];
    
    let closest_idx = 0;
    let min_dist = 10000;
    for (let i = 0; i < res_t.length; i++) {
        let dist = Math.abs(res_t[i] - t_curr);
        if (dist < min_dist) {
            min_dist = dist;
            closest_idx = i;
        }
    }
    
    source_dot.data['t'][0] = t_curr;
    source_dot.data['y'][0] = res_y[closest_idx];
    source_dot.change.emit();
    
    span_t.location = t_curr;
""")

slider_t.js_on_change('value', callback)

# Pre-roll Python
t_start = t_init
h_start = t_start - 1.0
h_end = t_start
for i, val_tau in enumerate(tau_vals):
    h_val = 1.0 if (val_tau >= h_start and val_tau <= h_end) else 0.0
    source_signals.data['h_shifted'][i] = h_val
    source_prod.data['prod'][i] = x_data[i] * h_val

layout = column(slider_t, p1, p2, p3, sizing_mode="scale_width")
show(layout)

Loading BokehJS ...